In [12]:
# Import Modules
import math
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from warnings import catch_warnings
from warnings import simplefilter
import scipy.stats as stats
import itertools

In [13]:
# objective function
def objective(x):
	model20 = [13.46458095,-198.47099612,75.47320401,153.0072275803083]
	model35 = [18.66491871,-320.58644811,115.4635245,227.21147521930675]
	result = [x[0]*model20[0] + x[1]*model20[1] + x[2]*model20[2] + model20[3],x[0]*model35[0] + x[1]*model35[1] + x[2]*model35[2] + model35[3]]
	return result

In [14]:
# evaluation function
def eval(y):
	return ((y[0] - 87.5)/87.5)**2 + ((y[1] - 76)/76)**2

In [15]:
# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter('ignore')
        return model.predict(X, return_std=True)

In [16]:
# Expected improvement acquisition function
def acquisition(X, Xsamples, model):
    # calculate the best surrgate score found so far
    yhat, _ = surrogate(model, X)
    score1 = np.zeros(len(yhat))
    for i in range(len(yhat)):
        score1[i] = eval(yhat[i])
    best = min(score1)

    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)
    score2 = np.zeros(len(mu))
    for i in range(len(mu)):
        score2[i] = eval(mu[i])
    # Calculate the expected improvement (EI)
    # Clip std to avoid division by zero
    std = np.clip(std, 1e-9, None)  # Replace None with a suitable upper bound if needed
    std2 = [i[0] for i in std]
    z = (score2 - best)**2 / std2
    ei = (score2 - best)**2 * stats.norm.cdf(z) + std2 * stats.norm.pdf(z)
    return ei

In [17]:
# optimize the acquisition function
def opt_acquisition(X, y, model):
    # random search, generate random samples
    Xs1 = 10 + 10*np.random.random(46)
    Xs2 = 2*np.random.random(46)
    Xs3 = -10 + 20*np.random.random(46)
    Xsamples = np.asarray(list(itertools.product(Xs1,Xs2,Xs3)))
    
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = np.argmin(scores)
    return Xsamples[ix]

In [19]:
# Sample Data
X = np.asarray([[0.6,0.125],[0.55,0.12],[0.65,0.1],[0.63,0.09],[0.625,0.07],[0.627,0.07]])
y = np.asarray([[1336.3,0.26],[1387.6,0.27338],[1307.7,0.34467],[1329.8,0.38731],[1346,0.50819],[1344.2,0.50819]])

X1 = [X[i][0] for i in range(len(X))]
X2 = [X[i][1] for i in range(len(X))]
y1 = [y[i][0] for i in range(len(y))]
y2 = [y[i][1] for i in range(len(y))]

# define the model
model = GaussianProcessRegressor()
model.fit(X,y)

# perform the optimization process
xp2 = []
yp2 = []
for i in range(100):
	# select the next point to sample
	x = opt_acquisition(X, y, model)
	# sample the point
	actual = objective(x)
	# summarize the finding
	est, _ = surrogate(model, [x])
	print(f'{i}step:>x = {x}, f()={est}, actual={actual}')
	# add the data to the dataset
	X = np.vstack((X, x))
	y = np.vstack((y,actual))
	# update the model
	model.fit(X, y)

	score3=np.zeros(len(y))
	for i in range(len(y)):
			score3[i] = eval(y[i])
	ix = np.argmin(score3)

	xp2.append(i)
	yp2.append(score3[ix])

plt.scatter(xp2,yp2)


GaussianProcessRegressor()

In [ ]:
# best result
score3=np.zeros(len(y))
for i in range(len(y)):
        score3[i] = eval(y[i])
ix = np.argmin(score3)
print(X[ix], y[ix])

plt.scatter(score3)

[15.84087571  1.21725429 -0.48785868] [87.88805091 76.31501939]


TypeError: scatter() missing 1 required positional argument: 'y'